## Explore whether LLMs can identify waiting

I'm curious to know how well an LLM can identify things that a patient might be waiting for from the notes. Below are a couple of examples of prompts that I used to extract this information. I found that a more detailed prompt asking for actions pending worked better

In [2]:
notebook_name = 'explore_waiting'

In [60]:
import os
import pandas as pd
import sys
import json

from pathlib import Path

from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# Import the variables that have been set in the init.py folder in the root directory
# These include a constant called PROJECT_ROOT which stores the absolute path to this folder
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import init
PROJECT_ROOT = os.getenv("PROJECT_ROOT")

# Add the src folder to sys path, so that the application knows to look there for libraries
sys.path.append(str(Path(PROJECT_ROOT) / 'src'))

In [8]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = Ollama(model="llama2", callback_manager=callback_manager)

### Load data from other experiments

In [6]:
# unstuctured data
with open('../src/data_exports/explore_llama_20240225_1202.json') as f:
    notes_data = json.load(f)

### Using llama to identify what patient is waiting for

In [67]:
template = """[INST]
<<SYS>>
You will be given a note or notes about a patient in hospital. 
From the notes, identify what the patient is currently waiting for.

Here is the information about the patient: 

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
# chain = prompt | llm

{'data' : notes_data[0]}

formatted_prompt = prompt.format_prompt(data = {'data' : notes_data[0]})
response = llm(formatted_prompt.to_string())

Based on the information provided in the notes, the patient is currently waiting for the results of a CT scan of the head to rule out any intracranial causes of their symptoms.

In [68]:
template = """[INST]
<<SYS>>
You will be given a note or notes about a patient in hospital. 
From the notes, identify what the patient is currently waiting for. There may be more than one thing they are waiting for.

Here is the information about the patient: 

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
# chain = prompt | llm

{'data' : notes_data[0]}

formatted_prompt = prompt.format_prompt(data = {'data' : notes_data[0]})
response = llm(formatted_prompt.to_string())

Based on the information provided in the notes, the patient is currently waiting for:

1. Admission to the hospital for further evaluation and management: The doctor has recommended admitting the patient to the hospital for a more detailed evaluation and treatment plan.
2. A CT scan of the head to rule out any intracranial causes of the patient's symptoms: The doctor has ordered a CT scan to help identify any underlying causes of the patient's headache, nausea, and vomiting.
3. A triage of pain medications to help manage the patient's headache: The doctor has provided the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache symptoms in the short term.
4. Closely monitoring the patient's vital signs and fluid status: The doctor is closely monitoring the patient's vital signs and fluid status to ensure that he stays hydrated and stable during his hospitalization.
5. Consideration of prescribing a preventive medication for the patient

In [66]:
template = """[INST]
<<SYS>>
You are a highly experienced emergency room nurse whose job is to minimise delays to patient care. 
A note has just been written after a doctor examined of the patient. 
From the note, identify what procedures, medications or other actions need to be taken immediate. 
List these actions succinctly in a to-do list for your colleagues. Do not justify why the actions are needed.
Limit your list only to things that would be done in hospital, and only items that the doctor is sure about.
Only use the information provided in the note. Do not give additional diagnoses. 

Here is the information about the patient: 

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
# chain = prompt | llm

{'data' : notes_data[0]}

formatted_prompt = prompt.format_prompt(data = {'data' : notes_data[0]})
response = llm(formatted_prompt.to_string())


Here is the list of actions that need to be taken immediate based on the information provided in the note:

1. Admit the patient to the hospital for further evaluation and management.
2. Order a CT scan of the head to rule out any intracranial causes of the patient's symptoms.
3. Provide the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache.
4. Monitor the patient's vital signs and fluid status closely, and provide appropriate fluids and electrolyte replacement as needed.

In [28]:
batch = []

for note_ in notes_data:
    batch.append({"data": note_})

In [29]:
outputs = chain.batch(batch)

Immediate actions needed:

1. Admit the patient to the hospital for further evaluation and management.
2. Order a CT scan of the head to rule out any intracranial causes of the patient's symptoms.
3. Provide the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache.
4. Monitor the patient's vital signs and fluid status closely, and provide appropriate fluids and electrolyte replacement as needed.Immediate actions required:

1. Admit the patient to the hospital for further observation and treatment.
2. Provide intravenous fluids to prevent dehydration.
3. Administer non-steroidal anti-inflammatory drugs (NSAIDs) for pain management.
4. Initiate a sumatriptan pain management plan.
5. Check the patient's blood work for any signs of electrolyte imbalances or other underlying conditions.Immediate actions required based on the information provided in the note:

1. Order urgent imaging studies (chest X-ray or CT scan) to confirm the presen

In [65]:
note_dict = {}

for i in range(0,len(batch)):
    key = list(notes_data[i].keys())[0]
    values = {'admission_note' : list(notes_data[i].values())[0], 
              'immediate actions' : outputs[i]}
    note_dict[key] = values


from utils.write_to_json import write_to_json
write_to_json(note_dict, notebook_name)






In [64]:
json.dump(note_dict)

TypeError: dump() missing 1 required positional argument: 'fp'